# Сопоставление шаблонов

На лекции мы познакомились с методом строк `split()`, который используется для разбиения строк. Очень часто во время разбиения строк заранее неизвестно, сколько фрагментов мы в итоге получим. В большинстве случаев такое поведение нас устраивает. Однако существуют случаи, когда нам важно знать, сколько сегментов в результате разбиения было получено, что это за сегменты, каковы их значения.

Яркий пример такого подхода к анализу фрагментов строки после разбиения - текстовые ролевые игры. Предположим, что мы разрабатываем как раз такую игру. Процесс игры состоит в обмене сообщениями с компьютером через терминал. Пользователь может вводить определенные команды, по типу `идти <направление>` для перемещения, `продать <предмет_1> [<предмет_2>, ..., <предмет_N>]` для продажи предметов, `выйти` для выхода из игры и т.д. В ответ на действия пользователя компьютер будет печатать в стандартный поток вывода различные сообщения, описывающие текущее состояние игрока и мира вокруг.

Видно, что команды, которые вводит пользователь, могут состоять из одного слова, как команда `выйти`, из двух слов, как команда  `идти <направление>`, или из произвольного числа слов, как команда `продать <предмет_1> [<предмет_2>, ..., <предмет_N>]`. Мы, как разработчики, должны учитывать этот факт, чтобы корректно обработать каждую команду. Вот как мы можем сделать это с помощью ветвления:

In [ ]:
unknown_command_msg_template = "Неизвестная команда: {command!r}"
directions = ("вперед", "назад", "вправо", "влево")

command = "идти вперед"
parts = command.split()

if len(parts) == 1:
    if parts[0] == "выйти":
        ...

    else:
        print(unknown_command_msg_template.format(command=parts[0]))

if len(parts) == 2:
    if parts[0] == "идти" and parts[-1] in directions:
        ...

    else:
        print(unknown_command_msg_template.format(command=command))

if len(parts) == 3:
    ...

Разумеется, этот пример не претендует на продуктовое качество. Однако, надеемся, вы уловили, насколько неудобна такая проверка. Нам приходится выполнять кучу повторяющихся действий, а большое количество кода дублируется. К счастью, в `Python 3.10` был добавлен более удобный инструмент для подобных проверок - сопоставление шаблонов с помощью конструкции `match-case`.

Итак, далее мы будем пытаться реализовать нашу текстовую игру с помощью `match-case`. Но начнем мы с цикла событий. 

## input и цикл событий

Цикл событий - это основа любой игры. Цикл событий - это условно бесконечный цикл, который прерывается с помощью специальной команды выхода, введенной пользователем. Каждая итерация цикла событий состоит из трех основных шагов:

- Пользователь вводит какую-либо команду
- Наша программа обрабатывает команду пользователя
- На экран выводится результат выполнения команды

После выполнения итерации цикла событий происходит или переход к очередной итерации, или завершение игры, если пользователь ввел соответствующую команду.

В нашей текстовой игре тоже будет цикл событий. Поскольку мы ограничены по времени, никакая сложная логика обработки команд реализована не будет. Вся обработка будет заключаться в выборе сообщения для печати на экране на основе введенной команды. Обработка будет выполняться с помощью методов строк и конструкции `match-case`. Вывод результата обработки команды будет происходить с помощью знакомой нам функции `print()`. Однако непонятно, что делать со вводом?

Для того, чтобы пользователь имел возможность ввода команд, мы будем использовать встроенную функцию `input()`. Функция `input()` позволяет считать текст, введенный в стандартный поток ввода, и сконструировать на его основе строку:

In [ ]:
name = input("Enter your name: ")
age = input("Enter your age: ")
hobbies = input("Enter your hobbies: ")

print(
    f"name: {name!r}",
    f"age: {age!r}",
    f"hobbies: {hobbies!r}",
    sep="\n",
)

Обратите внимание, что считывание происходит до появления первого символа перехода на новую строку. Также обратите внимание, что вне зависимости от введенного сообщения результатом выполнения функции `input()` всегда является строка. Даже если в стандартный поток ввода было введено число, в коде вы получите строку. Стоит быть особенно осторожными, если вы планируете использовать введенный результат для каких-либо математических вычислений:

In [ ]:
num = input("Enter non-zero number: ")
print(f"inversed num: {1 / num}")

В данном примере мы планировали обратить введенное число. Однако вместо успешного результата, мы получили `TypeError`, т.к. переменная `num` указывает на объект строкового типа данных, а деление чисел на строки не допускается. Чтобы избежать подобной ошибки, необходимо сконструировать объект требуемого типа данных явно:

In [ ]:
num = float(input("Enter non-zero number: "))
print(f"inversed num: {1 / num}")

Итак, зная о функции `input()`, реализуем первое приближения цикла событий:

In [ ]:
command_stop = "выйти"

while True:
    command = input("Ваш ход 🎲: ")

    if command == command_stop:
        break

print("🍺 Победа! 🍺")

Пока что ничего содержательного мы не сделали. Мы просто считываем команды пользователя в бесконечном цикле и завершаем работу программы, если пользователь ввел `выйти`. Да и `match-case` мы нигде не использовали. Поэтому давайте усложним нашу программу и добавим обработку перемещений.

## match-case: простейший вариант

In [ ]:
while True:
    command = input("Ваш ход 🎲: ")

    match command.split():
        case ["выйти"]:
            print("Игра окончена 💔")
            break

        case ["идти", direction]:
            print(f"Вы пошли {direction} 🚶")

Итак, в данном примере мы наконец-то воспользовались `match-case`. `match-case` - это специальная конструкция для сопоставления паттернов. В качестве паттернов могут выступать многие объекты. Паттерны описываются в `case`-блоках. Принцип работы `match-case` следующий:

- В заголовке `match` происходит "захват" значения некоторого объекта. Именно это значение и будет сопоставляться с различными шаблонами в `case`-блоках, перечисленных в теле инструкции `match`. В нашем случае в качестве захватываемого значения будет выступать результат выполнения метода `split()` для считанной команды.
- Захваченное значение будет последовательно сопоставляться с шаблонами, указанными в `case`-блоках. Шаблоны перебираются до тех пор, пока не будет встречен шаблон, которому удовлетворяет захваченное значение, или до тех пор, пока шаблоны не закончатся.
- Если значение соответствует шаблону, указанному в заголовке данного `case`-блока, будет выполнено тело этого `case`-блока. После выполнения тела `case`-блока происходит завершение выполнения инструкции `match-case`. Интерпретатор переходит к выполнению следующей инструкции.

В данном примере мы указали всего два шаблона: `["выйти"]` и `["идти", direction]`. Первый шаблон описывает любую последовательность, состоящую из одного элемента. Значение этого элемента - `"выйти"`. Т.е., фактически, первый `case`-блок используется для обработки команды `"выйти"`. Действительно, после применения к этой строке метода `split()` мы получим список `["выйти"]`, который удовлетворяет первому шаблону по определению. В качестве обработки этой команды мы печатаем сообщение о завершении игры и выходим из цикла событий.

Второй шаблон устроен хитрее. Как и в первом случае он описывает последовательность. Но на этот раз последовательность должна состоять из двух элементов. Значение первого элемента должно быть `"идти"`. А вот значение второго элемента может быть любым. В этом шаблоне мы просто связываем второй элемент с идентификатором `direction`. Затем этот идентификатор используется в теле `case`-блока, чтобы напечатать на экране направление движения пользователя. С помощью этого шаблона мы получаем возможность обрабатывать такие сообщения: `идти вперед`, `идти назад`, `идти прямо` и т.д.

Итак, начало положено. Теперь давайте улучшим нашу игру.

## match-case: случай по умолчанию

Очевидный минус текущей версии: если пользователь напечатает команду, которая не удовлетворяет ни одному шаблону, игра никак не отреагирует на это. Т.е. пользователь останется в неведении и будет думать, что игра зависла или сломалась. Давайте это исправим и добавим случай по умолчанию для тех команд, которые мы не умеем обрабатывать. В этом случае мы будем выводить сообщение о невозможности обработать команду пользователя:

In [ ]:
while True:
    command = input("Ваш ход 🎲: ")

    match command.split():
        case ["выйти"]:
            print("Игра окончена 💔")
            break

        case ["идти", direction]:
            print(f"Вы пошли {direction} 🚶")

        case _:
            print(f"Я не знаю такой команды {command!r} 😔")

Для обработки случая по умолчанию мы использовали специальную конструкцию `case _`. В контексте `match-case` идентификатор `_` имеет смысл случая по умолчанию, если используется как шаблон в заголовке `case`.

## match-case: subpatterns

Следующий момент, который нам нужно улучшить - ограничить направления перемещений. Сейчас пользователь может идти куда угодно. Мы бы хотели, чтобы пользователь перемещался только в четырех направлениях: вперед, назад, вправо, влево. Давайте ограничим значение элемента `direction`:

In [ ]:
while True:
    command = input("Ваш ход 🎲: ")

    match command.split():
        case ["выйти"]:
            print("Игра окончена 💔")
            break

        case ["идти", ("вперед" | "назад" | "вправо" | "влево") as direction]:
            print(f"Вы пошли {direction} 🚶")

        case _:
            print(f"Я не знаю такой команды {command!r} 😔")

С помощью круглых скобок и разделителей `|` мы перечислили допустимые значения для данного элемента захваченного значения. А с помощью оператора `as` мы по-прежнему имеем возможность связывать второй элемент захваченного значения с идентификатором `direction` и использовать его в теле `case`-блока.

## match-case: дополнительные условия

На самом деле перечислять значения направлений так, как мы это сделали в прошлом примере - не очень удобно. Гораздо удобнее завести какую-нибудь коллекцию с перечислением всех допустимых направлений. Тогда мы получим возможность использовать коллекцию направлений в других частях программы без дублирования кода. Однако непонятно, как это реализовать, сохранив проверку ограничений в паттерне. На самом деле это можно сделать с помощью дополнительных условий:

In [ ]:
directions = ("вперед", "назад", "вправо", "влево")

while True:
    command = input("Ваш ход 🎲: ")

    match command.split():
        case ["выйти"]:
            print("Игра окончена 💔")
            break

        case ["идти",  direction] if direction in directions:
            print(f"Вы пошли {direction} 🚶")

        case _:
            print(f"Я не знаю такой команды {command!r} 😔")

За описанием шаблона, после ключевого слова `if`, можно перечислить дополнительные условия, которые необходимо проверить. Обращаем ваше внимание, что в этих условиях могут фигурировать переменные, определенные во время проверки соответствия захваченного значения шаблону.

## match-case: случай по умолчанию для отдельной команды

Итак, теперь наша программа корректно обрабатывает команду перемещения. Команды формата `идти прямо` будут обработаны как неизвестные команды. С одной стороны, это логичная обработка, т.к. `прямо` не входит в число известных направлений. С другой стороны команда `идти прямо` - это скорее известная нам команда `идти` с неизвестной опцией `прямо`. Поэтому не совсем корректно было бы отвечать на эту команду сообщением `"Я не знаю такой команды ..."`. Корректнее было бы ответить как-то иначе, указав пользователю, что команда верная, но выбранная опция не поддерживается.

Сделать это можно, указав в шаблоне на месте второго параметра `_`: `["идти", _]`. Такой шаблон описывает последовательность из двух элементов. Первый элемент - `"идти"`, а второй элемент - любой объект, значение которого отличается от разрешенных направлений. Это суждение справедливо, если расположить паттерн `["идти", _]` после паттерна с обработкой известных направлений:

In [ ]:
directions = ("вперед", "назад", "вправо", "влево")

while True:
    command = input("Ваш ход 🎲: ")

    match command.split():
        case ["выйти"]:
            print("Игра окончена 💔")
            break

        case ["идти",  direction] if direction in directions:
            print(f"Вы пошли {direction} 🚶")

        case ["идти", _]:
            print("Сюда ходить нельзя 🚫")

        case _:
            print(f"Я не знаю такой команды {command!r} 😔")

## match-case: объединение паттернов

Теперь давайте добавим еще пару команд для взаимодействия с окружающим миром. Добавим команду `взять <объект>`, для того, чтобы брать объекты окружающего мира себе в инвентарь, и команду `выбросить <объект>`, чтобы избавляться от ненужных предметов в инвентаре. Эти команды по сути своей очень похожи, поэтому мы объединим их обработку в один кейс, используя `|` для объединения паттернов:

In [ ]:
directions = ("вперед", "назад", "вправо", "влево")

while True:
    command = input("Ваш ход 🎲: ")

    match command.split():
        case ["выйти"]:
            print("Игра окончена 💔")
            break

        case ["идти",  direction] if direction in directions:
            print(f"Вы пошли {direction} 🚶")

        case ["идти", _]:
            print("Сюда ходить нельзя 🚫")

        case ["взять" as action, obj] | ["выбросить" as action, obj]:
            action = action.replace("ть", "ли")
            print(f"Вы {action} {obj} 🎒")

        case _:
            print(f"Я не знаю такой команды {command!r} 😔")

## match-case: запаковка

Итак, теперь мы умеем брать и выбрасывать различные предметы. Однако у текущей реализации есть существенная проблема: чтобы взять или выбросить несколько вещей, нам придется выполнить соответствующую команду несколько раз подряд. Очевидно, это неудобно и создает плохой опыт пользователя. Поэтому давайте позволим пользователям брать несколько вещей и избавляться от нескольких вещей за одну команду. В реализации этой задумки нам поможет запаковка:

In [ ]:
directions = ("вперед", "назад", "вправо", "влево")

while True:
    command = input("Ваш ход 🎲: ")

    match command.split():
        case ["выйти"]:
            print("Игра окончена 💔")
            break

        case ["идти",  direction] if direction in directions:
            print(f"Вы пошли {direction} 🚶")

        case ["идти", _]:
            print("Сюда ходить нельзя 🚫")

        case ["взять" as action, *objs] | ["выбросить" as action, *objs]:
            action = action.replace("ть", "ли")
            objects = ", ".join(objs)
            print(f"Вы {action} {objects} 🎒")

        case _:
            print(f"Я не знаю такой команды {command!r} 😔")

## Дополнительно

В этом примере рассмотрены далеко не все особенности работы с `match-case`. Если вы хотите рассмотреть больше примеров использования этой составной инструкции, советуем ознакомиться с [официальным мануалом](https://peps.python.org/pep-0636/).

## Практика

Условия практических задач вы найдете [тут](https://github.com/EvgrafovMichail/python_mipt_dafe_tasks/blob/main/conditions/lesson05/tasks.md).